In [1]:
# Offline training of an end-to-end clip
# Encoder/decoder.
# Try to return to torch_emb=False
import sys
import os
sys.path.append(os.path.abspath("/Users/stefanhangler/Documents/Uni/Msc_AI/3_Semester/Seminar_Practical Work/Code.nosync/COATI"))

import torch.multiprocessing as mp
from coati.training.train_coati import train_autoencoder, do_args
import os
import inspect

from coati.data.dataset import COATI_dataset

In [2]:
def main():
    args = do_args()
    args.nodes = 1  # total num nodes.
    args.nr = 0  # rank of this node.
    # note args.gpus will default to the # gpus on this node.
    args.data_parallel = True

    args.test_frac = 0.02
    args.valid_frac = 0.0
    args.n_layer_e3gnn = 5
    args.n_hidden_e3nn = 256
    args.msg_cutoff_e3nn = 12.0
    args.n_hidden_xformer = 256
    args.n_embd_common = 256
    args.n_layer_xformer = 16
    args.n_head = 16
    args.max_n_seq = 250  # max the model can forward
    #    args.n_seq = 90 # max allowed in training.
    args.n_seq = 80  # max allowed in training.
    args.biases = True
    args.torch_emb = False
    args.norm_clips = True
    args.norm_embed = False
    args.token_mlp = True

    args.tokenizer_vocab = "mar"
    args.p_dataset = 0.2
    args.p_formula = 0.0
    args.p_fim = 0.0
    args.p_graph = 0.0
    args.p_clip = 0.9
    args.p_clip_emb_smi = 0.5
    args.p_randsmiles = 0.3
    args.batch_size = 160

    args.online = False  # Possible offline training of an end-to-end clip
    args.lr = 5.0e-4
    args.weight_decay = 0.1

    args.dtype = "float"
    args.n_epochs = 25
    args.clip_grad = 10
    args.test_interval = 2
    args.debug = False

    args.resume_optimizer = False
    # resume from checkpoint file
    # args.resume_document = ''

    args.ngrad_to_save = 2e6

    # output logs
    args.output_dir = "./logs/"
    # where to save model checkpoints
    args.model_dir = "./model_ckpts/"
    # where to save dataset cache
    args.data_dir = "/Volumes/PHILIPS/"
    args.model_filename = "coati_grande"


    COATI_dataset(cache_dir=args.data_dir).get_data_pipe()
    print(f"running on {args.gpus} gpus")
    #########################################################
    args.world_size = args.gpus * args.nodes
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "8899"
    #mp.spawn(train_autoencoder, nprocs=args.gpus, args=(args,))
    #########################################################


if __name__ == "__main__":
    main()


Warning... unparsed:  ['--f=/Users/stefanhangler/Library/Jupyter/runtime/kernel-v2-375301479vbUJ1yn9.json']
trying to open a raw datapipe for...
Will download ~340 GB of data to /Volumes/PHILIPS/ . This will take a while. Are you sure? [y/n] 

NoCredentialsError: Unable to locate credentials

In [ ]:
import boto3
from botocore.client import Config
from botocore.exceptions import NoCredentialsError

s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

try:
    s3.download_file('your-public-bucket', 'path/to/public/object', 'local_filename.ext')
    print("Download succeeded without credentials.")
except NoCredentialsError:
    print("Failed to download file without credentials.")
except Exception as e:
    print(f"An error occurred: {str(e)}")


In [5]:
import pickle
from coati.common.s3 import download_from_s3
import boto3
import tqdm

s3_resource = boto3.resource("s3")
bucket = s3_resource.Bucket("terray-public")

print(bucket.objects.all())



#nfiles = len(list(bucket.objects.filter(Prefix=bucket_dir)))

#print(nfiles)

bucket_dir = "datasets/coati_data/"
dest_dir = "/Volumes/PHILIPS/coati/" # absolute path

def copy_bucket_dir_from_s3(bucket_dir, dest_dir):
    s3_resource = boto3.resource("s3")
    bucket = s3_resource.Bucket("terray-public")
    nfiles = len(list(bucket.objects.filter(Prefix=bucket_dir)))
    if nfiles < 1:
        print(list(bucket.objects.filter(Prefix=bucket_dir)))
        raise Exception(f"empty_s3 {bucket_dir}")
    else:
        print(f"copying {nfiles} files from {bucket_dir} to {dest_dir}")
    for obj in tqdm(bucket.objects.filter(Prefix=bucket_dir), total=nfiles):
        if not os.path.exists(os.path.dirname(dest_dir + obj.key)):
            os.makedirs(os.path.dirname(dest_dir + obj.key))
        bucket.download_file(obj.key, dest_dir + obj.key)  # save to same path

copy_bucket_dir_from_s3(bucket_dir, dest_dir)

#download_from_s3("s3://terray-public/datasets/coati_data/")

# datasets are available as pickle files (list of dictionaries)
#with open("./datasets/delaney.pkl", "rb") as f:
#    delaney = pickle.load(f)

#delaney[0].keys()

s3.Bucket.objectsCollection(s3.Bucket(name='terray-public'), s3.ObjectSummary)


EndpointConnectionError: Could not connect to the endpoint URL: "https://terray-public.s3.Austria.amazonaws.com/?prefix=datasets%2Fcoati_data%2F&encoding-type=url"

In [17]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from botocore.handlers import disable_signing
import tqdm

def copy_bucket_dir_from_s3(bucket_dir, dest_dir):
    resource = boto3.resource('s3')
    resource.meta.client.meta.events.register('choose-signer.s3.*', disable_signing)

    bucket = resource.Bucket('terray-public')

    nfiles = len(list(bucket.objects.filter(Prefix=bucket_dir)))

    #if nfiles < 1:
        #print(list(bucket.objects.filter(Prefix=bucket_dir)))
       # raise Exception(f"empty_s3 {bucket_dir}")
    #else:
        #print(f"copying {nfiles} files from {bucket_dir} to {dest_dir}")
    for obj in tqdm(bucket.objects.filter(Prefix=bucket_dir), total=nfiles):
        if not os.path.exists(os.path.dirname(dest_dir + obj.key)):
            os.makedirs(os.path.dirname(dest_dir + obj.key))
        bucket.download_file(obj.key, dest_dir + obj.key)

# Usage
bucket_dir = "datasets/coati_data/"
dest_dir = "/Volumes/PHILIPS/coati/"
copy_bucket_dir_from_s3(bucket_dir, dest_dir)


ClientError: An error occurred (AccessDenied) when calling the ListObjects operation: Access Denied

In [7]:
import boto3

def download_public_file(bucket_name, object_key, download_path):
    s3_client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='', region_name='us-west-2')
    s3_client._request_signer.sign = (lambda *args, **kwargs: None)

    try:
        s3_client.download_file(bucket_name, object_key, download_path)
        print(f"Downloaded {object_key} to {download_path}")
    except Exception as e:
        print(f"Failed to download {object_key}: {e}")

# Usage
bucket_name = 'terray-public'
object_key = 'datasets/coati_data/1.pkl' 
download_path = 'path/to/store/file/1.pkl'
download_public_file(bucket_name, object_key, download_path)

Failed to download datasets/coati_data/1.pkl: [Errno 2] No such file or directory: 'path/to/store/file/1.pkl.4eE09bdE'


In [6]:
print(boto3.__version__)

1.9.251


In [3]:
import pickle
# load pickle file
with open("/Volumes/PHILIPS/coati/0.pkl", "rb") as f:
    data = pickle.load(f)

In [4]:
len(data)

20000

In [11]:
import boto3
import os

def list_pkl_files(bucket_name, prefix):
    """
    List all .pkl files in an S3 bucket folder.

    Args:
        bucket_name (str): The name of the S3 bucket.
        prefix (str): The prefix (folder path) in the bucket to search for files.

    Returns:
        list of str: A list of keys representing the .pkl files in the bucket.
    """
    s3_client = boto3.client('s3', aws_access_key_id='', aws_secret_access_key='', region_name='us-west-2')
    s3_client._request_signer.sign = (lambda *args, **kwargs: None)
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    pkl_files = [item['Key'] for item in response.get('Contents', []) if item['Key'].endswith('.pkl')]
    return pkl_files

def download_pkl_files(bucket_name, files, download_dir):
    """
    Download .pkl files from an S3 bucket to a local directory.

    Args:
        bucket_name (str): The name of the S3 bucket.
        files (list of str): List of file keys to download.
        download_dir (str): Local directory to save the downloaded files.

    Returns:
        None
    """
    s3 = boto3.client('s3')
    for file_key in files:
        local_filename = os.path.join(download_dir, os.path.basename(file_key))
        s3.download_file(Bucket=bucket_name, Key=file_key, Filename=local_filename)
        print(f"Downloaded {file_key} to {local_filename}")

# Usage
bucket_name = 'terray-public'
prefix = 'datasets/'
download_dir = './data'

# List .pkl files
pkl_files = list_pkl_files(bucket_name, prefix)
print("PKL files found:", pkl_files)


ClientError: An error occurred (AccessDenied) when calling the ListObjectsV2 operation: Access Denied